In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# !pip install torch torchvision torchaudio
# !pip install yolov5

In [28]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Hoặc yolov5m, yolov5l, yolov5x

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-21 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [29]:
# Đường dẫn đến tập dữ liệu
dataset_path = "/content/drive/MyDrive/txttoimage11/dataset"

In [30]:
import os

def analyze_images(dataset_path, model):
    """
    Phân tích ảnh trong tập dữ liệu và tạo nhãn.

    Args:
      dataset_path: Đường dẫn đến thư mục chứa tập dữ liệu.
      model: Mô hình YOLOv5.
    """
    for filename in os.listdir(dataset_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(dataset_path, filename)
            results = model(image_path)

            # Trích xuất đặc điểm đối tượng
            labels = []
            for *xyxy, conf, cls in results.xyxy[0]:
                label = model.names[int(cls)]
                labels.append(label)

            # Lưu nhãn vào file txt cùng tên với ảnh
            label_path = os.path.splitext(image_path)[0] + ".txt"
            with open(label_path, "w") as f:
                f.write(", ".join(labels))

# Gọi hàm phân tích ảnh
analyze_images(dataset_path, model)

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.am

In [31]:
# !pip install datasets -q

In [32]:
from torchvision import transforms

def preprocess(image):
    preprocess = transforms.Compose(
        [
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
            transforms.ToPILImage(),  # Chuyển đổi thành ảnh PIL
        ]
    )
    return preprocess(image)

In [42]:
import os
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

# Tải tập dữ liệu
dataset = load_dataset("imagefolder", data_dir=dataset_path)

# Khởi tạo pipeline Stable Diffusion
pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

# Đọc tất cả các nhãn trước
labels_dict = {}
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        label_path = os.path.join(dataset_path, os.path.splitext(filename)[0] + ".txt")
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                labels_dict[filename] = f.read()

# Xử lý tập dữ liệu
def preprocess_function(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["image"]]
    examples["pixel_values"] = pipe.feature_extractor(images=images, return_tensors="pt").pixel_values

    # Sử dụng trực tiếp nhãn từ YOLO
    labels = []
    for i, image in enumerate(examples["image"]):
        filename = os.path.basename(image_filenames[i]["filename"])
        label = labels_dict.get(filename, "")
        labels.append(label)

    examples["labels"] = labels
    return examples

dataset = dataset.map(preprocess_function, batched=True)


# Tham số huấn luyện
training_args = TrainingArguments(
    output_dir="path/to/save/model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-6,
    num_train_epochs=10,
    save_strategy="epoch",
    push_to_hub=False,
)

# Huấn luyện mô hình
trainer = Trainer(
    model=pipe,
    args=training_args,
    train_dataset=dataset["train"],
)

trainer.train()

# Lưu mô hình
pipe.save_pretrained("path/to/save/model")

Resolving data files:   0%|          | 0/82 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Map:   0%|          | 0/41 [00:00<?, ? examples/s]

NameError: name 'image_filenames' is not defined

In [ ]:
from flask import Flask, request, jsonify
from diffusers import StableDiffusionPipeline

app = Flask(__name__)

# Tải mô hình Stable Diffusion đã huấn luyện
pipe = StableDiffusionPipeline.from_pretrained("path/to/save/model")
pipe = pipe.to("cuda")

@app.route('/generate', methods=['POST'])
def generate():
    description = request.form['description']
    # Xử lý mô tả và tạo prompt (ví dụ: thêm "no background")
    prompt = f"{description}, no background"
    image = pipe(prompt).images[0]
    # Chuyển đổi image sang base64 hoặc lưu vào file
    # ...
    return jsonify({'image': image_base64})  # Trả về ảnh

if __name__ == '__main__':
    app.run()